# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK
using AtomsBuilder

## Feeding an AtomsBase AbstractSystem to DFTK

In this example we construct a bulk silicon system using the `bulk` function
from [AtomsBuilder](https://github.com/JuliaMolSim/AtomsBuilder.jl). This function
uses tabulated data to set up a reasonable starting geometry and lattice for bulk silicon.

In [2]:
system = bulk(:Si)

FlexibleSystem(Si₂, periodicity = TTT):
    cell_vectors      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")


By default the atoms of an `AbstractSystem` employ the bare Coulomb potential.
To employ pseudpotential models (which is almost always advisable for
plane-wave DFT) one employs the `pseudopotential` keyword argument in
model constructors such as `model_DFT`.
For example we can employ a `PseudoFamily` object
from the [PseudoPotentialData](https://github.com/JuliaMolSim/PseudoPotentialData.jl)
package. See its documentation for more information on the available
pseudopotential families and how to select them.

In [3]:
using PseudoPotentialData  # defines PseudoFamily

pd_lda_family = PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
model = model_DFT(system; functionals=LDA(), temperature=1e-3,
                  pseudopotentials=pd_lda_family)

Model(lda_x+lda_c_pw, 3D):
    lattice (in Bohr)    : [0         , 5.13061   , 5.13061   ]
                           [5.13061   , 0         , 5.13061   ]
                           [5.13061   , 5.13061   , 0         ]
    unit cell volume     : 270.11 Bohr³

    atoms                : Si₂
    atom potentials      : ElementPsp(Si, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Si.upf")
                           ElementPsp(Si, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Si.upf")

    num. electrons       : 8
    spin polarization    : none
    temperature          : 0.001 Ha
    smearing             : DFTK.Smearing.FermiDirac()

    terms                : Kinetic()
                           AtomicLocal()
                           AtomicNonlocal()
                           Ewald(nothing)
                           PspCorrection()
                           Hartree()
                           Xc(lda_x, lda_c_pw)
                   

Alternatively the `pseudopotentials` object also accepts a `Dict{Symbol,String}`,
which provides for each element symbol the filename or identifier
of the pseudopotential to be employed, e.g.

In [4]:
path_to_pspfile = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")[:Si]
model = model_DFT(system; functionals=LDA(), temperature=1e-3,
                  pseudopotentials=Dict(:Si => path_to_pspfile))

Model(lda_x+lda_c_pw, 3D):
    lattice (in Bohr)    : [0         , 5.13061   , 5.13061   ]
                           [5.13061   , 0         , 5.13061   ]
                           [5.13061   , 5.13061   , 0         ]
    unit cell volume     : 270.11 Bohr³

    atoms                : Si₂
    atom potentials      : ElementPsp(Si, "/home/runner/.julia/artifacts/966fd9cdcd7dbaba6dc2bf43ee50dd81e63e8837/Si.gth")
                           ElementPsp(Si, "/home/runner/.julia/artifacts/966fd9cdcd7dbaba6dc2bf43ee50dd81e63e8837/Si.gth")

    num. electrons       : 8
    spin polarization    : none
    temperature          : 0.001 Ha
    smearing             : DFTK.Smearing.FermiDirac()

    terms                : Kinetic()
                           AtomicLocal()
                           AtomicNonlocal()
                           Ewald(nothing)
                           PspCorrection()
                           Hartree()
                           Xc(lda_x, lda_c_pw)
                   

We can then discretise such a model and solve:

In [5]:
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921726366805                   -0.69    5.6    1.92s
  2   -7.926132486310       -2.36       -1.22    1.0    1.06s
  3   -7.926833770658       -3.15       -2.37    2.0    192ms
  4   -7.926861281201       -4.56       -3.00    3.2    198ms
  5   -7.926861647975       -6.44       -3.40    2.0    216ms
  6   -7.926861671294       -7.63       -3.87    1.4    554ms
  7   -7.926861679565       -8.08       -4.19    2.1    156ms
  8   -7.926861681779       -8.65       -4.97    1.1    139ms
  9   -7.926861681860      -10.09       -5.24    2.6    169ms
 10   -7.926861681871      -10.94       -6.05    1.2    141ms
 11   -7.926861681873      -11.94       -6.46    2.6    180ms
 12   -7.926861681873      -13.47       -7.26    1.5    145ms
 13   -7.926861681873   +  -15.05       -7.53    2.6    177ms
 14   -7.926861681873   +    -Inf       -7.95    1.0    138ms
 15   -7.

If we did not want to use
[AtomsBuilder](https://github.com/JuliaMolSim/AtomsBuilder.jl)
we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
which directly yields an AbstractSystem.

In [6]:
using AtomsIO
system = load_system("Si.extxyz");

Run the LDA calculation:

In [7]:
pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
model  = model_DFT(system; pseudopotentials, functionals=LDA(), temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921738270143                   -0.69    5.6    210ms
  2   -7.926132523619       -2.36       -1.22    1.0    131ms
  3   -7.926833547805       -3.15       -2.37    2.0    152ms
  4   -7.926861262539       -4.56       -3.01    2.9    193ms
  5   -7.926861655079       -6.41       -3.43    2.4    168ms
  6   -7.926861674168       -7.72       -3.97    1.4    145ms
  7   -7.926861678684       -8.35       -4.10    2.0    159ms
  8   -7.926861681782       -8.51       -4.67    1.0    141ms
  9   -7.926861681832      -10.30       -4.98    1.6    151ms
 10   -7.926861681858      -10.59       -5.30    1.5    155ms
 11   -7.926861681872      -10.84       -5.89    1.1    144ms
 12   -7.926861681873      -12.45       -6.93    2.1    161ms
 13   -7.926861681873      -13.34       -6.34    3.0    189ms
 14   -7.926861681873   +    -Inf       -6.43    1.0    143ms
 15   -7.

The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [8]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
model  = model_DFT(system; pseudopotentials, functionals=LDA(), temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921732979674                   -0.69    5.8    241ms
  2   -7.926135842130       -2.36       -1.22    1.0    122ms
  3   -7.926837343259       -3.15       -2.37    2.0    166ms
  4   -7.926864503508       -4.57       -3.00    3.0    195ms
  5   -7.926865069053       -6.25       -3.41    2.1    164ms
  6   -7.926865085172       -7.79       -3.94    1.2    132ms
  7   -7.926865089064       -8.41       -4.03    1.9    146ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [9]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodicity = TTT):
    cell_vectors      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [10]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, pseudopotentials)
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodicity = TTT):
    cell_vectors      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")
